In [ ]:
# 2025/7/24
# zhangzhong
# https://docs.pytorch.org/docs/stable/notes/amp_examples.html

In [ ]:
# https://docs.pytorch.org/docs/stable/notes/amp_examples.html#typical-mixed-precision-training

In [ ]:
# Working with Unscaled Gradients
# https://docs.pytorch.org/docs/stable/notes/amp_examples.html#working-with-unscaled-gradients

# Gradient Clipping
scaler = GradScaler()

for epoch in epochs:
    for input, target in data:
        optimizer.zero_grad()
        with autocast(device_type='cuda', dtype=torch.float16):
            output = model(input)
            loss = loss_fn(output, target)
        scaler.scale(loss).backward()

        # Unscales the gradients of optimizer's assigned params in-place
        scaler.unscale_(optimizer)

        # Since the gradients of optimizer's assigned params are unscaled, clips as usual:
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

        # optimizer's gradients are already unscaled, so scaler.step does not unscale them,
        # although it still skips optimizer.step() if the gradients contain infs or NaNs.
        scaler.step(optimizer)

        # Updates the scale for next iteration.
        scaler.update()

In [ ]:
# Working with Multiple GPUs
# https://docs.pytorch.org/docs/stable/notes/amp_examples.html#distributeddataparallel-one-gpu-per-process
# torch.nn.parallel.DistributedDataParallel’s documentation recommends one GPU per process for best performance. 
# In this case, DistributedDataParallel does not spawn threads internally, so usages of autocast and GradScaler are not affected.

model = MyModel()
dp_model = nn.DataParallel(model)

# Sets autocast in the main thread
with autocast(device_type='cuda', dtype=torch.float16):
    # dp_model's internal threads will autocast.
    output = dp_model(input)
    # loss_fn also autocast
    loss = loss_fn(output)